## GK2A (https://nmsc.kma.go.kr/homepage/html/base/cmm/selectPage.do?page=static.satllite.introGk2a)

In [ ]:
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy
!pip uninstall shapely
!pip install shapely --no-binary shapely
!pip install netCDF4

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import numpy as np
from netCDF4 import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nc_path='/content/drive/MyDrive/Colab Notebooks/'
nc_fname='gk2a_ami_le1b_ir105_ea020lc_202104010000.nc'

nc=Dataset(nc_path+nc_fname,'r', format='netcdf4')
print(nc)

In [ ]:
ncdata=nc['image_pixel_values'] #data=nc.variables['image_pixel_values']와 같음
print(ncdata)

In [ ]:
data=nc.variables['image_pixel_values'][:,:]
print(data)

In [ ]:
data_1d=data.flatten()
plt.hist(data_1d, bins=range(np.min(data),np.max(data), 100))
plt.show()

In [ ]:
mask = 0b0001111111111111 #13bit mask
ipixel_process_masked=np.bitwise_and(data,mask)
np.bitwise_and(13, 17) 

#숫자 13은 00001101 로 표시됩니다 . 마찬가지로 17은 00010001 로 표시됩니다 .
#따라서 13과 17의 비트 AND는 000000001 또는 1입니다.

In [ ]:
CT_path='/content/drive/MyDrive/Colab Notebooks/ct_table/'
AL_postfix='_con_alb.txt'
BT_postfix='_con_bt.txt'

channel=ncdata.getncattr('channel_name')
print(channel)

conversion_table=np.loadtxt(CT_path+channel+BT_postfix,'float64')
data_converted=conversion_table[ipixel_process_masked]
data_converted.shape, np.min(data_converted), np.max(data_converted)

In [ ]:
plt.hist(data_converted.flatten(), bins=range(200, 301, 1))

In [ ]:
plt.imshow(data, cmap='gray')
plt.show()

In [ ]:
plt.imshow(data_converted, cmap='Greys')
plt.show()

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
geo_name='gk2a_latlon_ea_2000.bin'

with open(nc_path+geo_name,'rb') as f:
    nx, ny =  3000, 2600
    latlon = np.fromfile(f, dtype='float', count=nx*ny*2)

print(latlon.shape)

latlon=np.reshape(latlon, (ny,nx,2))

lat_arr=latlon[:,:,0]
lon_arr=latlon[:,:,1]

np.min(lat_arr[:,0]), np.min(lat_arr[:,1]), np.max(lat_arr), np.min(lon_arr), np.max(lon_arr)

In [ ]:
fig = plt.figure(figsize=(10,10))

p = ccrs.LambertConformal(central_longitude=126, central_latitude=38.0,globe=None)
ax = plt.axes(projection=p)
cf1 = ax.pcolormesh(lon_arr, lat_arr, data_converted, cmap='Greys', transform=ccrs.PlateCarree())
ax.coastlines()

ax.set_extent([110,140,20,50])
ax.set_title("GK2A_Lv1B_"+channel, size='xx-large')

cbar = plt.colorbar(cf1, orientation="vertical", pad=0.1,shrink=0.8)
cbar.set_label('K', size='x-large')

gl=ax.gridlines(draw_labels=True,x_inline=False, y_inline=False)
gl.xlabels_top = False
gl.ylabels_left = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 12}
gl.ylabel_style = {'size': 12}
plt.show()